We consider the graph marked as $g_4$ in the paper [FGG].

https://arxiv.org/pdf/1411.4028


In [2]:
import os, sys

SAGE_ROOT = "/usr/local/lib/sage"
SAGE_LOCAL = os.path.join(SAGE_ROOT, "local")
PYVER = f"{sys.version_info.major}.{sys.version_info.minor}"

# Candidate site-packages for Sage; add those that exist
candidates = [
    os.path.join(SAGE_LOCAL, f"lib/python{PYVER}/site-packages"),
    os.path.join(SAGE_LOCAL, f"var/lib/sage/venv-python{PYVER}/lib/python{PYVER}/site-packages"),
    os.path.join(SAGE_ROOT, "src"),
]
for p in candidates:
    if os.path.isdir(p):
        sys.path.insert(0, p)

os.environ["SAGE_ROOT"] = SAGE_ROOT
os.environ["SAGE_LOCAL"] = SAGE_LOCAL

from sage.all import *

MAXIMA_SHARE  = '@SAGE_MAXIMA_SHARE@'
MAXIMA_FAS = '/usr/local/lib/sage/local/lib/ecl/maxima.fas'
from sage.libs.ecl import EclObject, ecl_eval
ecl_eval("(require 'maxima \"{}\")".format(MAXIMA_FAS))

;;; Loading "/usr/local/lib/sage/local/lib/ecl/maxima.fas"
;;; Loading #P"/usr/lib/x86_64-linux-gnu/ecl-21.2.1/sb-bsd-sockets.fas"
;;; Loading #P"/usr/lib/x86_64-linux-gnu/ecl-21.2.1/sockets.fas"


<ECL: ("SB-BSD-SOCKETS" "SOCKETS" "MAXIMA")>

In [3]:
def matrix_C(nr_vertices, edge_list):
    """Return the cost matrix C (diagonal) counting cut edges for each bitstring.
    Base ring is SR to be compatible with symbolic unitaries.
    """
    sequences = [list(map(int, format(i, f'0{nr_vertices}b'))) for i in range(2**nr_vertices)]
    C = zero_matrix(SR, 2**nr_vertices, 2**nr_vertices)
    for i, s in enumerate(sequences):
        C[i, i] = -len([e for e in edge_list if s[e[0]] != s[e[1]]])
    return C


def matrix_edge(nr_vertices, edge):
    """Return the diagonal matrix for a single edge: 1 if the edge is cut by the bitstring, else 0.
    Base ring is SR to match symbolic operations.
    """
    sequences = [list(map(int, format(i, f'0{nr_vertices}b'))) for i in range(2**nr_vertices)]
    C_edge = zero_matrix(SR, 2**nr_vertices, 2**nr_vertices)
    a, b = edge
    for i, s in enumerate(sequences):
        C_edge[i, i] = -1 if s[a] != s[b] else 0
    return C_edge


In [4]:
def UB_tensor(nr_bits, b):
    """
    Construct UB = exp(-i * b * B) for B = sum_i X_i
    via tensor product: UB = ⊗_{k=1}^n exp(-i b X).
    This is symbolic in b and efficient.
    Base ring is SR (Symbolic Ring) to interoperate with other symbolic matrices.
    """
    X = matrix(SR, [[0, 1],
                    [1, 0]])
    I2 = identity_matrix(SR, 2)
    # exp(-i b X) = cos(b) * I2 - i * sin(b) * X
    U1 = cos(b) * I2 - I * sin(b) * X
    U = U1
    for _ in range(nr_bits - 1):
        U = U.tensor_product(U1)
    return U

def expC(C, c):
    Cdiag = C.diagonal()
    UCdiag = [exp(-I*c*Cdiag[i]) for i in range(C.nrows())]
    return diagonal_matrix(UCdiag)


In [5]:
def function_graph(nr_vertices, edge_list, edge):
    b = var('b', domain=RR)
    c = var('c', domain=RR)

    C = matrix_C(nr_vertices, edge_list)
    C_edge = matrix_edge(nr_vertices, edge)
    UB = UB_tensor(nr_vertices, b)
    UB_dag = UB.conjugate().transpose()
    
    UC = expC(C, c)
    
    BCB = UC.conjugate().transpose()*UB_dag*C_edge*UB
    s = (2.0**nr_vertices)**(-1/2)*vector([1.0]*2**nr_vertices)
    assert abs(s.norm() - 1) < 10.0**-5
    f = s*BCB*s
    return f.real().simplify()

In [6]:
edges4 = [[0,1],[1,2],[2,3],[3,0],[1,3]]
edge4 = [1,3]

g4 = Graph(edges4)
f4 = function_graph(4, edges4, edge4)
g4.show()

Graphics object consisting of 10 graphics primitives


In [7]:
edges5 = [[0,1],[0,2],[1,2],[1,3],[2,4]]
edge5 = [1,2]
g5 = Graph(edges5)
f5 = function_graph(5, edges5, edge5)
g5.show()

Graphics object consisting of 11 graphics primitives


In [8]:
edges6 = [[0,2],[1,2],[2,3],[3,4],[3,5]]
edge6 = [2,3]
g6 = Graph(edges6)
f6 = function_graph(6, edges6, edge6)
g6.show()

Graphics object consisting of 12 graphics primitives


In [16]:
Graph('S}KGGKB?G?_A?B?B??O?A??K??_??g??[')

Graph on 20 vertices

In [15]:
f6

-0.03125*cos(b)^12*cos(5*c) - 0.125*cos(b)^12*cos(4*c) - 0.1875*cos(b)^12*cos(3*c) - 0.125*cos(b)^12*cos(2*c) - 0.03125*cos(b)^12*cos(c) - 0.0625*cos(b)^10*cos(5*c)*sin(b)^2 - 0.375*cos(b)^10*cos(4*c)*sin(b)^2 - 0.875*cos(b)^10*cos(3*c)*sin(b)^2 - 1.0*cos(b)^10*cos(2*c)*sin(b)^2 - 0.5625*cos(b)^10*cos(c)*sin(b)^2 + 0.03125*cos(b)^8*cos(5*c)*sin(b)^4 - 0.375*cos(b)^8*cos(4*c)*sin(b)^4 - 1.8125*cos(b)^8*cos(3*c)*sin(b)^4 - 2.875*cos(b)^8*cos(2*c)*sin(b)^4 - 1.96875*cos(b)^8*cos(c)*sin(b)^4 + 0.125*cos(b)^6*cos(5*c)*sin(b)^6 - 0.25*cos(b)^6*cos(4*c)*sin(b)^6 - 2.25*cos(b)^6*cos(3*c)*sin(b)^6 - 4.0*cos(b)^6*cos(2*c)*sin(b)^6 - 2.875*cos(b)^6*cos(c)*sin(b)^6 + 0.03125*cos(b)^4*cos(5*c)*sin(b)^8 - 0.375*cos(b)^4*cos(4*c)*sin(b)^8 - 1.8125*cos(b)^4*cos(3*c)*sin(b)^8 - 2.875*cos(b)^4*cos(2*c)*sin(b)^8 - 1.96875*cos(b)^4*cos(c)*sin(b)^8 - 0.0625*cos(b)^2*cos(5*c)*sin(b)^10 - 0.375*cos(b)^2*cos(4*c)*sin(b)^10 - 0.875*cos(b)^2*cos(3*c)*sin(b)^10 - 1.0*cos(b)^2*cos(2*c)*sin(b)^10 - 0.5625*cos(b)^2

In [10]:
def maximize_f(f, b_min, b_max, c_min, c_max, b_steps=360, c_steps=180, refine_factor=10):
    """
    Numerically maximize f(b, c) over [b_min, b_max] x [c_min, c_max].
    Returns (f_max, b_argmax, c_argmax).
    """
    f_fast = fast_callable(f, vars=[b, c], domain=RDF)
    def eval_f(bb, cc):
        return f_fast(float(bb), float(cc))

    db = (b_max - b_min)/b_steps
    dc = (c_max - c_min)/c_steps
    max_val = -infinity
    b_star = None
    c_star = None
    bb_vals = srange(b_min, b_max, db, include_endpoint=True)
    cc_vals = srange(c_min, c_max, dc, include_endpoint=True)
    for bb in bb_vals:
        for cc in cc_vals:
            val = eval_f(bb, cc)
            if val > max_val:
                max_val = val
                b_star, c_star = bb, cc
    # local refinement around the coarse maximizer
    db_fine = db/refine_factor
    dc_fine = dc/refine_factor
    b_lo = max(b_min, b_star - 2*db)
    b_hi = min(b_max, b_star + 2*db)
    c_lo = max(c_min, c_star - 2*dc)
    c_hi = min(c_max, c_star + 2*dc)
    bb_vals_fine = srange(b_lo, b_hi, db_fine, include_endpoint=True)
    cc_vals_fine = srange(c_lo, c_hi, dc_fine, include_endpoint=True)
    for bb in bb_vals_fine:
        for cc in cc_vals_fine:
            val = eval_f(bb, cc)
            if val > max_val:
                max_val = val
                b_star, c_star = bb, cc
    return {'max_val':RR(max_val), 'b':RR(b_star), 'c':RR(c_star)}

# Example usage:
# f_max, b_argmax, c_argmax = maximize_f(f4, 0, 2*pi, 0, pi)
# print("max f(b,c) ≈", f_max)
# print("at b ≈", b_argmax, "c ≈", c_argmax)

In [11]:
# given a cubic graph, we compute the number of 
# - isolated triangles
# - crossed squares
# - the third type of neighbourhood

def number_of_neighbors(graph):

    edge_list = []
    for edge in graph.edges():
        neigh_j = graph.neighbors(edge[0])
        neigh_k = graph.neighbors(edge[1])
        neigh_jk = set(neigh_j+neigh_k)
        edge_list.append(len(neigh_jk))

    counts = {4:0, 5:0, 6:0}
    for val in edge_list:
        if not val in [4,5,6]:
            raise('value not in range')
            
        counts[val] += 1
    return counts

def optimal_angles_graph(graph):

    nr_edges_of_type = number_of_neighbors(graph)
    func = nr_edges_of_type[4]*f4+nr_edges_of_type[5]*f5+nr_edges_of_type[6]*f6 
    return maximize_f(-func, 0, 2*pi, 0, pi)


In [12]:
g = graphs.RandomRegular(3,20)
g.edges()

[(0, 17, None), (0, 3, None), (0, 9, None), (1, 5, None), (1, 7, None), (1, 13, None), (2, 8, None), (2, 12, None), (2, 15, None), (3, 8, None), (3, 13, None), (4, 16, None), (4, 19, None), (4, 9, None), (5, 6, None), (5, 9, None), (6, 16, None), (6, 10, None), (7, 16, None), (7, 15, None), (8, 15, None), (10, 17, None), (10, 18, None), (11, 18, None), (11, 19, None), (11, 14, None), (12, 13, None), (12, 14, None), (14, 19, None), (17, 18, None)]

In [13]:
optimal_angles_graph(g)

{'max_val': 15.2498915551595, 'b': 5.85208898193699, 'c': 0.0663225115757845}

In [14]:
# 3D surface of the magnitude of f over b,c in [-pi, pi]
surf = plot3d(lambda bb, cc: f.subs({b: bb, c: cc}), (b, 0, 2*pi), (c, 0, pi),
              mesh=True, color='red', plot_points=(200,200))
surf.save('plot.png')

# contour plot of the magnitude of f on the same domain
# cont = contour_plot(lambda bb, cc: f.subs({b: bb, c: cc}).n().real(), (b, -pi, pi), (c, -pi, pi),
#                    cmap='red')
# show(cont)

NameError: name 'f' is not defined